In [ ]:
def análisis_compacto(n, i, a, ruta, A, B, C, D, h, dis, pro):

        cargar_datos_2(n = n, i = i, ruta=ruta, a = a)

        #uso los dos extremos de la parte lineal para ajustar

        voltajes_intervalos_a = np.concatenate([voltajes1_a[A:B], voltajes1_a[C:D]])
        tiempos_intervalos_a = np.concatenate([tiempos1_a[A:B], tiempos1_a[C:D]])

        #Cuando se vea la recta, hago el ajuste:

        x_fit_a =  np.linspace(np.min(tiempos2_a), np.max(tiempos2_a), len(tiempos2_a), endpoint = True)

        param_iniciales = [0.01, 0]
        popt_0, pcov_0 = curve_fit(lineal, tiempos_intervalos_a,voltajes_intervalos_a, p0=param_iniciales, sigma = error_Ch1_a, absolute_sigma=True)
        incertidumbre_0 = np.sqrt(np.diag(pcov_0))

        tabla = pd.DataFrame({
                'Los parametros óptimos son': popt_0,
                'Los errores de los parámetros son': incertidumbre_0,
                })
        print(tabla)


        t_espectro_a = tiempos1_a[B:C] #así ya está recortado
        espectro_a = (voltajes1_a - lineal(x_fit_a, *popt_0))[B:C] 

        #inertimos, para graficar la absorción en función del tiempo
        #Y normalizamos para que el pico de mayor amplitud valga 1
        espectro_a_inv = espectro_a * (-1)
        espectro_a_norm = espectro_a / np.max(espectro_a_inv)


        #filtrado
        señal_suavizada_a = savgol_filter(espectro_a_norm, window_length=11, polyorder=2)


        #damos vuelta el espectro
        espectro_suave_a = señal_suavizada[::-1]


        #encuentro el primer y último pico

        peaks, props = find_peaks(
        espectro_suave_a,
        height= h,        # altura mínima del pico
        distance= dis,      # distancia mínima en puntos entre picos
        prominence= pro    # "prominencia" mínima (cuán destacado es el pico)
        )

        # obtener primer y último pico real
        x_first, x_last = t_espectro[peaks][0], t_espectro[peaks][-1]
        print("Primer pico:", x_first)
        print("Último pico:", x_last)

        # reescaleo los tiempos a las frecuencias 
        # Fórmula de reescalado
        freq_relativas = (t_espectro - x_first) / (x_last - x_first) * (7.649 - 0) + 0 # le pongo el cero por si lo quiero cambiar después
        freq_absolutas = (t_espectro - x_first) / (x_last - x_first) * (frecuencias_abs[1] - frecuencias_abs[0]) + frecuencias_abs[0]


        graficar_6_subplots(tiempos1_a, voltajes1_a, tiempos_intervalos_a, voltajes_intervalos_a, t_espectro_a, espectro_a, t_espectro_a, señal_suavizada_a, t_espectro_a, espectro_suave_a, freq_relativas, espectro_suave_a)


        #creo las variables globales

        globals()[f'espectro_suave_{a}'] = espectro_suave_a
        globals()[f'f_rel_{a}'] = freq_relativas
        globals()[f'f_abs_{a}'] = freq_relativas



        

In [ ]:
        if norm: #si queremos normalizar
                espectro_a_norm = espectro_a_pos / np.max(espectro_a_pos)

                #filtrado
                señal_suavizada_a = savgol_filter(espectro_a_norm, window_length=11, polyorder=2)
        else:
                #filtrado
                señal_suavizada_a = savgol_filter(espectro_a_pos, window_length=11, polyorder=2)
